# Face Crawl from csv file with pandas / icrawler

In [ ]:
import pandas as pd
from icrawler.builtin import GoogleImageCrawler
from icrawler.builtin import BingImageCrawler
from pathlib import Path

mitFilter=True
filters = dict(type='photo') # either photo, face, clipart, linedrawing, animated
howmany= 10
names=pd.read_csv('/home/base/Documents/Git/Projekte/Face-celeb-rec/Top 1000 Actors and Actresses.csv', encoding = "ISO-8859-1")
#names.head()
#names.columns
#names.Name

#subset=names.Name[0:3]
#print(subset)
subset=names.Name

path='~/Documents/Git/Projekte/Face-celeb-rec/Data/'

#for keyword in subset:
#    crawler = GoogleImageCrawler(
#        parser_threads=5,
#        downloader_threads=5,
#         storage={path : 'Celebs/{}'.format(keyword)}
#        storage={'root_dir': 'Celebs/{}'.format(keyword)}
#    )
    
for keyword in subset:
    crawler = BingImageCrawler(
        parser_threads=5,
        downloader_threads=5,
        storage={'root_dir': 'Celebs/{}'.format(keyword)}
    )    
    if mitFilter==True:
        crawler.crawl(keyword=keyword, filters=filters,max_num=howmany, min_size=(500, 500))
    else:
        crawler.crawl(keyword=keyword, max_num=howmany, min_size=(500, 500))


2020-01-21 23:18:31,507 - INFO - icrawler.crawler - start crawling...
2020-01-21 23:18:31,508 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-01-21 23:18:31,509 - INFO - feeder - thread feeder-001 exit
2020-01-21 23:18:31,509 - INFO - icrawler.crawler - starting 5 parser threads...
2020-01-21 23:18:31,512 - INFO - icrawler.crawler - starting 5 downloader threads...
2020-01-21 23:18:31,758 - INFO - parser - parsing result page https://www.bing.com/images/async?q=Robert De Niro&first=0&qft=+filterui:photo-photo
2020-01-21 23:18:31,797 - INFO - downloader - skip downloading file 000001.jpg
2020-01-21 23:18:31,799 - INFO - downloader - skip downloading file 000002.jpg
2020-01-21 23:18:31,799 - INFO - downloader - skip downloading file 000003.jpg
2020-01-21 23:18:31,800 - INFO - downloader - skip downloading file 000004.jpg
2020-01-21 23:18:31,801 - INFO - downloader - skip downloading file 000005.jpg
2020-01-21 23:18:31,802 - INFO - downloader - skip downloading file 000006.j

# Face crawl and save in in train Validate and test folder

In [ ]:
import os
import sys
import time
import argparse
import logging
import face_recognition
import pandas as pd

from icrawler.builtin import GoogleImageCrawler


names=pd.read_csv('/home/base/Documents/Git/Projekte/Face-celeb-rec/Top 1000 Actors and Actresses.csv', encoding = "ISO-8859-1")


logger = logging.getLogger('crawler')


def main(keyword, max_num=20, image_dir=None):
    if not keyword:
        raise ValueError('Keyword parameter is required.')

    keyword = keyword.strip()

    logger.info('Crawler run for: %s' % keyword);
    print('Crawler run for: %s' % keyword);

    if image_dir is None:
        image_dir = 'data/training_img/%s' % keyword.replace(" ", "_")

    image_dir = os.path.expanduser(image_dir)
    os.makedirs(image_dir, exist_ok=True)

    google_crawler = GoogleImageCrawler(feeder_threads=10, parser_threads=10, log_level=logging.DEBUG,
                                        downloader_threads=25, storage={'root_dir': image_dir})
    filters = dict(type='photo') # I find photo more accurate than 'face'
    start = time.time()
    google_crawler.crawl(keyword=keyword, filters=filters, offset=0, max_num=max_num,
                         min_size=(512, 512), max_size=(2000,2000), file_idx_offset=0)

    for (rootDir, dirNames, filenames) in os.walk(image_dir):
        for f in filenames:
            filename = os.path.join(rootDir, f)
            image = face_recognition.load_image_file(filename)
            face_locations = face_recognition.face_locations(image)
            # I just keep images with 1 single face
            if len(face_locations) != 1:
                os.remove(filename)
    end = time.time()
    logger.info("Time elapsed: %.2f seconds", end - start)


def parse_arguments(argv):
    parser = argparse.ArgumentParser()
    parser.add_argument('-k', "--keyword", type=str, required=True,
                        help='Keyword for searching')
    parser.add_argument("--max_num", type=int, default=20,
                        help='Max number of images to download')
    parser.add_argument("--image_dir", type=str, default=None,
                        help='Output folder for downloaded images')
    return parser.parse_args(argv)


if __name__ == '__main__':
    args = parse_arguments(sys.argv[1:])
    main(args.keyword, args.max_num, args.image_dir)